In [32]:
import matplotlib.pyplot as plt
import os
import numpy as np
import math
import subprocess

> **Note:** This cell below is optional.  
> The bash script often printed extra comments, which cluttered the output.  
> Running this cell simply helps keep the output cleaner.

In [33]:
import os

# Strip out the exported bash functions that break Jupyter subshells
for key in list(os.environ):
    if key.startswith("BASH_FUNC_"):
        del os.environ[key]

print("✅ Cleaned exported bash functions. Subshells will be quiet now.")


✅ Cleaned exported bash functions. Subshells will be quiet now.


> Sometimes you may want to run code in a different directory than where this `.ipynb` notebook is located.  
> The cell below sets the working directory for everything you run after executing it.

In [34]:
# set working directory for all future cells
os.chdir("/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone")
# and check with bash command pwd - note the exclamation mark at the beginning
!pwd
# or in python
os.getcwd()

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone


'/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone'

> **Important:**  
> These scripts are tailored for **ACCRE** and load modules available on ACCRE at Vanderbilt University.
> 
> **Option 1 – Adjust for your HPC:**  
> Edit the `module load` lines to match the Amber modules you would like to use
> **Option 2 – Source Amber.sh:**  
> If your cluster doesn’t use modules, remove those lines and source the AMBER environment , e.g.:
> ```bash
> source /sb/apps/amber22/amber.sh
> ```
> This ensures the correct AMBER version is used on any cluster.


In [35]:
import os, subprocess

def load_amber_env():
    """Load AmberTools + Amber environment into the Jupyter kernel."""
    bash_cmd = """
    module purge
    module load StdEnvACCRE/2023
    module load gcc/12.3 openmpi/4.1.5 cuda/12.2
    module load ambertools/25.0
    module load scipy-stack/2025a
    module load amber/24
    env
    """
    env_output = subprocess.check_output(["bash", "-c", bash_cmd], text=True)
    for line in env_output.splitlines():
        if "=" in line:
            k, v = line.split("=", 1)
            os.environ[k] = v
    print("✅ Amber environment loaded into Jupyter kernel.")


### 🧩 Covalent System Preprocessing (PPARγ + EEY, PDB: 6ONI)

To prepare the covalent complex for MD simulations, the protein and ligand were carefully processed, merged, and parameterized:

1. **Protein Preparation**  
   - Started from `6ONI.pdb` and removed the ligand.  
   - Uploaded the apo protein to the **H++ server** to generate a **protonated structure**.  

2. **Ligand + Cys83 Extraction and Merging**  
   - In PyMOL, loaded `6ONI.pdb` again and kept **only Cys83 and ligand EEY**.  
   - Covalently bonded EEY to **Cys83**, forming the thioether linkage
   - Edited names so that Cys83 and EEY are both named as T007 and have the same number - 83
   - Saved this fragment as a separate PDB to create a hybrid ligand  
   - Reopened on pymol and added **hydrogens** (using PyMOL `h_add` or Chimera’s *AddH* tool) to protonate the ligand.  
   - saved this **hybrid protonated ligand**

3. **Capping & Atom Editing in Chimera**  
   - Opened the ligand–Cys83 fragment in UCSF Chimera.  
   - Added **hydrogen atoms** to cap the N- and C- termini.  
   - Converted the terminal hydrogens → carbons → methyl groups  
     (important because charge derivation requires closed, neutral termini).  

4. **Charge Derivation**  
   - Uploaded this file to the **R.E.D. Server** to derive **RESP charges** for the covalently linked ligand.  

5. **Complex Build for LEaP**  
   - Deleted **Cys83** from the protonated protein structure prepared in Step 1.  
   - Merged the **hybrid protonated ligand** fragment from Step 2 with the deleted **Cys83** structure.  
   - Covalently bonded T007 to the protein backbone at **Cys83** (amide N and carbonyl C linked to residues 82 and 84).  
   - Verified amidation and carbonyl connectivity to ensure proper backbone reconstruction.  
   - Edited ligand to place **hybrid protonated ligand** in the same **chain A**.  
   - Renumbered the ligand residue to **83**, sorted the structure in PyMOL, and saved as `PPARg_T007_combined1.pdb`.  

---

> Run the next cell to load the **AmberTools + Amber environment** into your Jupyter kernel.  
> This ensures all subsequent commands use the correct modules and environment variables.


In [ ]:
load_amber_env()

### 💊 Ligand Preparation

Run **Antechamber** to convert the RESP/RED-derived `.mol2` file  
into an AMBER-compatible `.ac` file with atom types and connectivity.

In [ ]:
%cd ligand_prep
# Define atom types for AMBER. We skip `-at amber -c bcc` to preserve RESP/RED charges,
# which are more reproducible and consistent than AMBER's bcc-derived charges.
!antechamber -fi mol2 -i ../RED_server_output/Data-R.E.D.Server/Mol_m1/Mol-sm_m1-c1.mol2  -fo ac -o t007a.ac

/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/ipykernel/2025a/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone/ligand_prep


The following modules were not unloaded:
  (Use "module --force purge" to unload all):

  1) CCconfig        6)  ucx/1.14.1         11) flexiblas/3.3.1
  2) gentoo/2023     7)  libfabric/1.18.0   12) imkl/2023.2.0
  3) gcccore/.12.3   8)  pmix/4.2.4         13) StdEnvACCRE/2023
  4) gcc/12.3        9)  ucc/1.2.0
  5) hwloc/2.9.1     10) openmpi/4.1.5
Lmod Warning:
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: scipy-stack/2023b
(required by: plumed/2.9.0, ambertools/25.0)
-------------------------------------------------------------------------------




The following have been reloaded with a version change:
  1) ipykernel/2023b => ipykernel/2025a
  2) scipy-stack/2023b => scipy-stack/2025a



✅ Amber environment loaded into Jupyter kernel.
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for mol2 File --
   Status: pass
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
/bin/bash

Lmod Warning:
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: scipy-stack/2025a
(required by: amber/24)
-------------------------------------------------------------------------------




Inactive Modules:
  1) arrayfire/3.9.0     2) cudnn/8.9.5.29     3) glfw/3.3.8     4) spdlog/1.11.0

Due to MODULEPATH changes, the following have been reloaded:
  1) ambertools/25.0     2) plumed/2.9.0     3) ucc-cuda/1.2.0     4) ucx-cuda/1.14.1

The following have been reloaded with a version change:
  1) cuda/12.2 => cuda/12.6
  2) cudacore/.12.2.2 => cudacore/.12.6.2
  3) ipykernel/2025a => ipykernel/2023b
  4) nccl/2.18.3 => nccl/2.26.2
  5) scipy-stack/2025a => scipy-stack/2023b



sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'



### 🔎 Compare MOL2 and AC Files

This cell helps compare the original RESP/RED-derived `.mol2` file with the `t007a.ac` file generated by Antechamber:

- Verifies that all atoms from the `.mol2` file are present in the `.ac` file.  
- Checks that atom names match between the two files.  
- Notice that **AC_Type** (in `.ac`) uses GAFF atom types (lowercase),  
  while `Mol2_Type` (in `.mol2`) uses Amber-style capitalized types.  

A combined table is displayed for easy side-by-side comparison,  
and any mismatched atom names are flagged.


In [37]:
#Comparing mol2 file and t007a.ac file that was generated, we see that all atoms were added 
#But the type of all the atoms is changed to gaff (small letter in AC_Type column compared to capital in Mol2_type in the output table)
#Good idea to keep the forcefields for the backbone CYS as amber type so figure out the backbone atoms on pymol or any other visualization software
import pandas as pd

# === File paths ===
mol2_file = "../RED_server_output/Data-R.E.D.Server/Mol_m1/Mol-sm_m1-c1.mol2"
ac_file   = "t007a.ac"

# --- Helper to parse MOL2 ---
def parse_mol2_atoms(file):
    atoms = []
    read_atoms = False
    with open(file) as f:
        for line in f:
            if line.startswith("@<TRIPOS>ATOM"):
                read_atoms = True
                continue
            if line.startswith("@<TRIPOS>"):
                read_atoms = False
            if read_atoms:
                parts = line.split()
                if len(parts) >= 6:
                    atom_no = int(parts[0])
                    atom_name = parts[1]
                    atom_type = parts[5]  # GAFF/Amber type
                    atoms.append((atom_no, atom_name, atom_type))
    return pd.DataFrame(atoms, columns=["AtomNo", "AtomName", "Mol2_Type"])

# --- Helper to parse AC file ---
def parse_ac_atoms(file):
    atoms = []
    with open(file) as f:
        for line in f:
            if line.startswith("ATOM"):
                parts = line.split()
                atom_no = int(parts[1])
                atom_name = parts[2]
                atom_type = parts[-1]  # Last column is type
                atoms.append((atom_no, atom_name, atom_type))
    return pd.DataFrame(atoms, columns=["AtomNo", "AtomName", "AC_Type"])

# --- Parse both files ---
df_mol2 = parse_mol2_atoms(mol2_file)
df_ac   = parse_ac_atoms(ac_file)

# --- Merge by atom number ---
merged = pd.merge(df_mol2, df_ac, on="AtomNo", suffixes=("_mol2", "_ac"))

# Flag if names mismatch
merged["NameMatch"] = merged["AtomName_mol2"] == merged["AtomName_ac"]

# Display nicely in Jupyter
from IPython.display import display
display(merged)

# Print summary of mismatches
if not merged["NameMatch"].all():
    mismatches = merged[~merged["NameMatch"]]
    print("⚠️ WARNING: Atom names mismatched between MOL2 and AC!")
    display(mismatches)
else:
    print("✅ All atom names match between MOL2 and AC.")


,AtomNo,AtomName_mol2,Mol2_Type,AtomName_ac,AC_Type,NameMatch
0,1,C1,CT,C1,c3,True
1,2,H11,H1,H11,h1,True
2,3,H1,H1,H1,h1,True
3,4,H13,H1,H13,h1,True
4,5,N,NT,N,n3,True
5,6,H2,H,H2,hn,True
6,7,CA,CX,CA,c3,True
7,8,HA,H1,HA,h1,True
8,9,C,C,C,c,True
9,10,O,O,O,o,True


✅ All atom names match between MOL2 and AC.


### 🧬 Revert Cys Backbone to Amber, Keep Ligand on GAFF

Since **Cys83 is part of the protein backbone**, this step sets its atom types back to **Amber force fields**,  
while leaving the rest of the ligand atoms as **GAFF**.

- `cys_atoms` defines the Cys83 backbone atom names.  
- Each matching atom in `t007a.ac` is updated with the capitalized Amber type from the `.mol2` file.  
- The edited file is saved as `t007_edit.ac`.  

This ensures that **Amber parameters are used for the protein backbone**  
and **GAFF parameters remain for the ligand**, resulting in a consistent hybrid system.


In [38]:
#cys_atoms defines atom names that belong to the CYS backbone and reverts to the capital name found in mol2 file 
#Now these capital letters tell amber to look for amber forcefields 
import pandas as pd

# === Your CYS atoms from PyMOL ===
cys_atoms = ["SG","CB","HB1","HB2","CA","N","H2","HA","O","C"]

# === Load mapping from the merged DataFrame ===
# If you already have 'merged' DataFrame from previous cell, reuse it.
# Otherwise re-load it from CSV or regenerate.
mapping = dict(zip(merged["AtomName_ac"], merged["Mol2_Type"]))

input_ac = "t007a.ac"
output_ac = "t007_edit.ac"

new_lines = []
with open(input_ac) as f:
    for line in f:
        if line.startswith("ATOM"):
            parts = line.split()
            atom_name = parts[2]
            atom_type = parts[-1]  # last column is type
            if atom_name in cys_atoms:
                new_type = mapping.get(atom_name, atom_type)
                parts[-1] = new_type
                line = "{:<6s}{:>6s} {:<4s} {:<4s} {:<4s}{:>6s}{:>8s}{:>10s}{:>10s}{:>10s}\n".format(
                    parts[0], parts[1], parts[2], parts[3], parts[4],
                    parts[5], parts[6], parts[7], parts[8], parts[-1]
                )
        new_lines.append(line)

with open(output_ac, "w") as f:
    f.writelines(new_lines)

print(f"✅ Edited AC file written to {output_ac}")


✅ Edited AC file written to t007_edit.ac


### ✅ Sanity Check for Edited AC File

This cell compares the original `t007a.ac` and the modified `t007_edit.ac` side by side:

- Verifies that atom names are unchanged.  
- Confirms that **Cys83 backbone atom types** were successfully reverted to Amber types.  
- Adds a `TypeChanged` flag to show which atoms were modified.  

> **Note:** If any atom still has type `NT`, manually change it to `N`  
> (since `NT` is not defined in `parm.dat` and can cause issues in LEaP).


In [39]:
# Sanity check confirms that the atom names were updated (prints 'True').
# Note: atom 5 currently has type 'NT' — we will manually change this to 'N'.
# 'NT' is not defined in parm.dat and can cause issues; 'N' is the correct type for amide nitrogen.

import pandas as pd

def read_ac_file(ac_file):
    atoms = []
    with open(ac_file) as f:
        for line in f:
            if line.startswith("ATOM"):
                parts = line.split()
                atoms.append([int(parts[1]), parts[2], parts[-1]])
    return pd.DataFrame(atoms, columns=["AtomNo", "AtomName", "AC_Type"])

# Read original and edited AC files
orig_df = read_ac_file("t007a.ac")
edit_df = read_ac_file("t007_edit.ac")

# Merge side-by-side on AtomNo
compare_df = orig_df.merge(edit_df, on="AtomNo", suffixes=("_old", "_new"))

# Add a flag for differences
compare_df["TypeChanged"] = compare_df["AC_Type_old"] != compare_df["AC_Type_new"]

# Show full comparison
compare_df


,AtomNo,AtomName_old,AC_Type_old,AtomName_new,AC_Type_new,TypeChanged
0,1,C1,c3,C1,c3,False
1,2,H11,h1,H11,h1,False
2,3,H1,h1,H1,h1,False
3,4,H13,h1,H13,h1,False
4,5,N,n3,N,NT,True
5,6,H2,hn,H2,H,True
6,7,CA,c3,CA,CX,True
7,8,HA,h1,HA,H1,True
8,9,C,c,C,C,True
9,10,O,o,O,O,True


### 📝 Create the `t007.mc` File (Methyl Cap Removal)

This cell writes the **`t007.mc` file**, which tells AMBER:

- Which atoms to **omit** (the temporary methyl caps added for charge derivation).  
- How to reattach the ligand residue to the **main protein chain** (head and tail connection points).  
- Sets the overall residue charge to **0.0**.

This ensures that when LEaP loads the ligand, it removes the caps and correctly connects T007–Cys83 back into the protein backbone.


In [40]:
%%bash
# Create this file to tell AMBER where to remove the methyl caps we added
# for forcefield generation, and how to attach the residue back to the main chain.
cat > t007.mc <<'EOF'
HEAD_NAME N
TAIL_NAME C
MAIN_CHAIN CA
OMIT_NAME C1
OMIT_NAME H11
OMIT_NAME H1
OMIT_NAME H13
OMIT_NAME C2
OMIT_NAME H21
OMIT_NAME H22
OMIT_NAME H23
PRE_HEAD_TYPE C
POST_TAIL_TYPE N
CHARGE 0.0
EOF

echo "✅ t007.mc created"


bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'


✅ t007.mc created


### 📦 Generate PREPIN File for LEaP

Run `prepgen` to create the **`t007.prepin`** file, which will be loaded in **LEaP**:

- Uses the edited `.ac` file (`t007_edit.ac`) with corrected atom types.  
- Applies the methyl cap removal and connection rules from `t007.mc`.  
- Assigns the ligand residue name as **T00**.

This PREPIN file defines the ligand’s topology and charges from R.E.D. server in a format AMBER can use.


In [41]:
#Run this to create the prepin file which will be used by telap 
load_amber_env()
!prepgen -i t007_edit.ac -o t007.prepin -m t007.mc -rn T00

bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unex

✅ Amber environment loaded into Jupyter kernel.
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'

PRE_HEAD_TYPE is     C
POST_TAIL_TYPE is     N
Net charge of truncated molecule is     0.00
HEAD_ATOM      5    N
TAIL_ATOM      9    C
MAIN_CHAIN     1    5    N
MAIN_CHAIN     2    7   CA
MAIN_CHAIN     3    9    C
OMIT_ATOM      1    1   C1
OMIT_ATOM      2    2  H11
OMIT_ATOM      3    3   H1
OMIT_ATOM      4    4  H13
OMIT_ATOM      5   11   C2
OMIT_ATOM      6   12  H21
OMIT_ATOM      7   13  H22
OMIT_ATOM      8   14  H23
Number of mainchain atoms (including h

Replace `NT` with `N` in `t007.prepin` to fix Amber’s incorrect atom type assignment.  


In [42]:
#Now 4 cells above this cell we said that we need to remove NT and replace it with N, this is doing just that 
!sed -i 's/\bNT\b/N/g' t007.prepin && echo "✅ NT → N replaced in t007.prepin"

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
✅ NT → N replaced in t007.prepin


⚙️ Run `parmchk2` on `t007.prepin` to generate **`frcmod.t007`**, referencing `parm10.dat` (Amber force fields).  
This ensures the **Cys backbone** uses Amber parameters, while the ligand section may still have `ATTN` lines  
because its atoms use GAFF types — we will fix these missing parameters in the next cells.  


In [ ]:

load_amber_env()
!parmchk2 -i t007.prepin -f prepi -o frcmod.t007 -p $AMBERHOME/dat/leap/parm/parm10.dat -a Y

bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unex

✅ Amber environment loaded into Jupyter kernel.
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'


⚙️ Remove `ATTN` lines from `frcmod.t007` to create **`frcmod1.t007`** (Amber parameters for the Cys backbone),  
then run `parmchk2` again to generate **`frcmod2.t007`** with GAFF parameters for the ligand.  
In LEaP, load `frcmod2.t007` first (ligand) and `frcmod1.t007` second (backbone) to get a complete parameter set.  


In [ ]:

!grep -v "ATTN" frcmod.t007 > frcmod1.t007 # Strip out ATTN lines
!parmchk2 -i t007.prepin -f prepi -o frcmod2.t007

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'


Display the last line of `prepin.inpcrd` to find the simulation box dimensions generated by `tleap`.

In [46]:
!tail -n 1 prepin.inpcrd

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
  87.7250100  87.7250100  87.7250100 109.4712190 109.4712190 109.4712190


Use `molarity.perl` (from the `Scripts` folder) to calculate the amount of KCl to add for a **50 mM solution**  
based on the given box dimensions:


In [47]:
#perl molarity.perl {conc in M} {pasted coordinates from above}
!perl ../Scripts/molarity.perl 0.05 87.7250100  87.7250100  87.7250100 109.4712190 109.4712190 109.4712190 

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
 MOLARITY =    0.050
 Box size =   87.725   87.725   87.725  109.471  109.471  109.471
 Volume = 519694.852

   15.648 molecules are necessary to make a molarity of   0.05 M



The molarity calculation output indicated that ~15.6 molecules are needed for a 0.05 M solution so we will add 16 K+ and 16 Cl- to the tleap.in file

Update the `tleap.in` file to adjust the number of ions added (K⁺ and Cl⁻) based on the molarity calculation:


In [49]:
# Replace K+ 4 -> K+ 16 and Cl- 0 -> Cl- 16 in tleap input file
!sed -i 's/#addions complex K+ 4/addions complex K+ 16/; s/#addions complex Cl- 0/addions complex Cl- 16/' tleapd.sh

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'


Sanity check - confirm that the intended changes were made 

In [50]:
!tail 10 tleapd.sh

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
tail: cannot open '10' for reading: No such file or directory
==> tleapd.sh <==
addions complex Cl- 0

solvateoct complex TIP3PBOX 10.0 iso

#addions complex K+ 16
#addions complex Cl- 16

saveamberparm complex prepin.prmtop prepin.inpcrd
savepdb       complex prepin.pdb
quit


In [ ]:
#Confirm that this runs now 
!tleap -s -f tleapd.in